In [1]:
import pandas as pd

In [40]:
#데이터 불러오기 및 크기 확인
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("훈련용 데이터 크기:", train_data.shape)
print("테스트용 데이터 크기:", test_data.shape)



훈련용 데이터 크기: (17480, 16)
테스트용 데이터 크기: (15081, 15)


In [41]:
#결측치 제거 함수

def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col


missing_col = check_missing_col(train_data)
print(missing_col)

train_data.dropna(axis=0,subset=['workclass','occupation','native.country'],inplace = True)

결측치가 있는 컬럼은: workclass 입니다
해당 컬럼에 총 1836 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: occupation 입니다
해당 컬럼에 총 1843 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: native.country 입니다
해당 컬럼에 총 583 개의 결측치가 존재합니다.
[['workclass', dtype('O')], ['occupation', dtype('O')], ['native.country', dtype('O')]]


In [42]:

x_train = train_data.drop(['id', 'target'], axis=1)
y_train = train_data['target']

test_data=test_data.drop(['id'], axis=1)


print("훈련용 데이터:", x_train.shape)
print("테스트용 라벨:", y_train.shape)

훈련용 데이터: (15081, 14)
테스트용 라벨: (15081,)


In [22]:
print("훈련용 데이터:", x_train.shape)
print("테스트용 라벨:", y_train.shape)
print("결과 확인용 라벨:",test_data.shape)

훈련용 데이터: (15080, 14)
테스트용 라벨: (15080,)
결과 확인용 라벨: (15081, 14)


In [43]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            #dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

x_train = label_encoder(x_train, make_label_map(x_train))

#제출용 라벨링 데이터 변환
test_data = label_encoder(test_data, make_label_map(test_data))

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train,Y_test = train_test_split(x_train, y_train, random_state=22)

In [50]:
#knn 최근접 이웃 분석
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, Y_train)

KNeighborsClassifier()

In [51]:
print("테스트 정확도: {:.3f}".format(clf.score(X_test, Y_test)))

테스트 정확도: 0.763


In [56]:
#로지스틱 회귀
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=0.01).fit(X_train, Y_train)

print("테스트 정확도: {:.3f}".format(logreg.score(X_test, Y_test)))


테스트 정확도: 0.792


In [61]:
#결정 트리
from sklearn.tree import DecisionTreeClassifier

max_depth=[2,4,8,12]

for i in max_depth : 

    tree = DecisionTreeClassifier(max_depth=i, random_state=0)
    tree.fit(X_train, Y_train)
    print("깊이",i, " 훈련세트 정확도: {:.3f}".format(tree.score(X_train, Y_train)))
    print("깊이",i," 테스트 정확도: {:.3f}".format(tree.score(X_test, Y_test)))

깊이 2  훈련세트 정확도: 0.826
깊이 2  테스트 정확도: 0.815
깊이 4  훈련세트 정확도: 0.840
깊이 4  테스트 정확도: 0.830
깊이 8  훈련세트 정확도: 0.859
깊이 8  테스트 정확도: 0.837
깊이 12  훈련세트 정확도: 0.893
깊이 12  테스트 정확도: 0.827


In [62]:
tree = DecisionTreeClassifier(max_depth=8, random_state=0)
tree.fit(X_train, Y_train)
print("깊이",i, " 훈련세트 정확도: {:.3f}".format(tree.score(X_train, Y_train)))
print("깊이",i," 테스트 정확도: {:.3f}".format(tree.score(X_test, Y_test)))

print("특성 중요도:\n", tree.feature_importances_)
    

깊이 12  훈련세트 정확도: 0.859
깊이 12  테스트 정확도: 0.837
특성 중요도:
 [0.05400141 0.00471263 0.01451146 0.00278002 0.22173479 0.39361079
 0.02118163 0.00514252 0.00071486 0.00188837 0.18839113 0.05203551
 0.03373379 0.00556108]


In [67]:
#랜덤 포레스트 트리 분석
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons

max_depth=[100,200,400,800,1200]

for i in max_depth : 

    forest = RandomForestClassifier(n_estimators=100, random_state=8)
    forest.fit(X_train, Y_train)

    print("깊이",i, " 훈련세트 정확도: {:.3f}".format(forest.score(X_train, Y_train)))
    print("깊이",i, " 테스트 정확도: {:.3f}".format(forest.score(X_test, Y_test)))

깊이 100  훈련세트 정확도: 1.000
깊이 100  테스트 정확도: 0.848
깊이 200  훈련세트 정확도: 1.000
깊이 200  테스트 정확도: 0.848
깊이 400  훈련세트 정확도: 1.000
깊이 400  테스트 정확도: 0.848
깊이 800  훈련세트 정확도: 1.000
깊이 800  테스트 정확도: 0.848
깊이 1200  훈련세트 정확도: 1.000
깊이 1200  테스트 정확도: 0.848


In [68]:
#그레이디언트 부스팅 회귀 트리 분석
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, Y_train)

print("깊이", " 훈련세트 정확도: {:.3f}".format(gbrt.score(X_train, Y_train)))
print("깊이", " 테스트 정확도: {:.3f}".format(gbrt.score(X_test, Y_test)))

깊이  훈련세트 정확도: 0.868
깊이  테스트 정확도: 0.851


In [74]:
max_depth=[1,5,10,20,50,100]
learning_rate=[0.01, 0.1, 1, 10]

max_accur =0;
max_accur_test=0;
depth =0;
learning_rates = 0;
for i in max_depth : 
    for j in learning_rate : 
        gbrt = GradientBoostingClassifier(random_state=0, max_depth= i, learning_rate= j)
        gbrt.fit(X_train, Y_train)

        #print("깊이", " 훈련세트 정확도: {:.3f}".format(gbrt.score(X_train, Y_train)))
        #print("깊이", " 테스트 정확도: {:.3f}".format(gbrt.score(X_test, Y_test)))  
        temp =gbrt.score(X_test, Y_test)
        if(max_accur_test <temp):
            max_accur = gbrt.score(X_train, Y_train)
            max_accur_test = temp
            depth = i;
            learning_rates = j;
            
print("최고치", depth, "learning_rate: ", learning_rates)
print(max_accur, " ", max_accur_test)
            

최고치 5 learning_rate:  0.1
0.8900972590627763   0.8631662688941926


In [75]:
#1일차 제출용 그레디언트 부스팅 회귀 트리
gbrt = GradientBoostingClassifier(random_state=0, max_depth= 5, learning_rate= 0.1)
gbrt.fit(x_train, y_train)



GradientBoostingClassifier(max_depth=5, random_state=0)

In [76]:
gbrt.predict(test_data)

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [77]:
result = pd.DataFrame({'target': gbrt.predict(test_data)})

In [78]:
result.to_csv('day1_result.csv')

In [79]:
# SVM 분석
from sklearn.svm import SVC

svc =SVC()
svc.fit(X_train, Y_train)

print("깊이", " 훈련세트 정확도: {:.3f}".format(svc.score(X_train, Y_train)))
print("깊이", " 테스트 정확도: {:.3f}".format(svc.score(X_test, Y_test)))

깊이  훈련세트 정확도: 0.778
깊이  테스트 정확도: 0.781


In [81]:
min_on_training =X_train.min(axis=0)
range_on_training = (X_train - min_on_training).max(axis=0)

X_train_scaled = (X_train - min_on_training) / range_on_training
X_test_scaled = (X_test - min_on_training) / range_on_training

svc =SVC()
svc.fit(X_train_scaled, Y_train)
print("깊이", " 훈련세트 정확도: {:.3f}".format(svc.score(X_train_scaled, Y_train)))
print("깊이", " 테스트 정확도: {:.3f}".format(svc.score(X_test_scaled, Y_test)))

깊이  훈련세트 정확도: 0.844
깊이  테스트 정확도: 0.834


In [85]:
svc =SVC(C=50)
svc.fit(X_train_scaled, Y_train)
print("깊이", " 훈련세트 정확도: {:.3f}".format(svc.score(X_train_scaled, Y_train)))
print("깊이", " 테스트 정확도: {:.3f}".format(svc.score(X_test_scaled, Y_test)))

깊이  훈련세트 정확도: 0.866
깊이  테스트 정확도: 0.840


In [91]:
#에이다 부스트
#머신러닝 알고리즘 테스트 마지막
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=2000, random_state=45)
ada.fit(X_train, Y_train)
print("훈련세트 정확도: {:.3f}".format(ada.score(X_train, Y_train)))
print("테스트 정확도: {:.3f}".format(ada.score(X_test, Y_test)))

훈련세트 정확도: 0.875
테스트 정확도: 0.864
